<div id = 3 style="color:white;
           display:fill;
           border-radius:20px;
           background-color:#b02222;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 6rem;
              color:white;
          text-align:center;
          margin:10 auto;
          ">
    ITERATION 10: Identify, Isolate and Utilize more popular Key Features


</h2>
</div>

In [11]:
import json

with open('../z_envs/_envs.json') as f:
    env_vars = json.loads(f.read())
print(env_vars)

run_env = env_vars['notebook_environment']

if run_env not in ['colab', 'gradient', 'cloud']:
    cloud_run = False
else:
    cloud_run = True
    import sys
    import os

    module_path = os.path.abspath(os.path.join('..', '..'))
    if module_path not in sys.path:
        sys.path.append(module_path)
        
    %pip install missingno

from functions_0__common_20221116 import get_columns
from functions_b__get_the_data_20221116 import set_csv_directory, get_combined_dataset
from functions_d1__prepare_cleanse_data_20221116 import add_supplements, tidy_dataset
from functions_d2__transform_enrich_data_20221116 import feature_engineer

{'notebook_environment': 'gradient', 'use_gpu': True, 'debug_mode': False, 'quick_mode': False, 'quick_override_cv_splits': 2, 'quick_override_n_iter': 10, 'quick_override_n_jobs': 3}
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
import missingno as msno

import warnings

In [13]:
VERSION = '10'

warnings.filterwarnings("ignore")

In [14]:
LABEL = 'Price'

columns, booleans, floats, categories, custom, wildcard = get_columns(version=VERSION)

print(colored(f"features", "blue"), "-> ", columns)
print(colored(f"+ ", "blue"), "-> ", wildcard)

columns.insert(0, LABEL)
original_columns = columns
print(colored(f"label", "green"), "-> ", LABEL)

features ->  ['bedrooms', 'bathrooms', 'nearestStation', 'location.latitude', 'location.longitude', 'latitude_deviation', 'longitude_deviation', 'tenure.tenureType']
+  ->  ['feature__']
label ->  Price


In [21]:
IN_COLAB = False

filename = f'df_listings_v{VERSION}.csv'
remote_pathname = f'https://raw.githubusercontent.com/jayportfolio/capstone_streamlit/main/data/final/{filename}'

df_pathname_raw = f'../../data/source/{filename}'
#df_pathname_tidy = f'../../data/final/{filename}'

df_pathname = df_pathname_raw if not cloud_run else remote_pathname
print(df_pathname)
df_orig = pd.read_csv(df_pathname, on_bad_lines='error', index_col=0)

df_orig['sharedOwnership.sharedOwnership'].head()

https://raw.githubusercontent.com/jayportfolio/capstone_streamlit/main/data/final/df_listings_v10.csv


KeyError: 'sharedOwnership.sharedOwnership'

In [18]:
df_prep = add_supplements(df_orig, version=VERSION)
df_prep = feature_engineer(df_prep, version=VERSION)

KeyError: 'sharedOwnership.sharedOwnership'

In [ ]:
updated_columns = False
columns = original_columns

for each in df_prep.columns:
    for prefix in wildcard:
        if each.startswith(prefix):
            columns.append(each)
            updated_columns = True

if updated_columns:
    print(colored(f"columns updated to:", "yellow"), "-> ", columns)
df_prep

In [ ]:
#print(df.keyFeatures.str.len().describe())
#df.keyFeatures.str.lower().str.contains('garden')
keyword = 'garden'
print(df_prep.keyFeatures.str.lower().str.contains(keyword).sum())
df_prep.keyFeatures.str.lower().str.contains(keyword)
df_prep.keyFeatures.str.lower().str.index("]")

In [ ]:
dailmail = ['garden','central heating','parking','off road','shower','cavity wall insulation','wall insulation','insulation','insulat','dining room','garage','en-suite','en suite']
common_knowledge = ['penthouse','balcony']
ideal_home = ['double-glazing','double glazing','off-road parking','security','patio','underfloor heating','marble']
discarded = ['signal','secure doors','secure door','outdoor lighting','bathtub','neighbours',]

#keywords = ['']
keywords = []
keywords.extend(dailmail)
keywords.extend(common_knowledge)
keywords.extend(ideal_home)
for keyword in keywords:
    print(keyword, df_prep.keyFeatures.str.lower().str.contains(keyword).sum())

In [ ]:
#df = feature_engineer(df, version=VERSION)

In [ ]:
import re
spice_df = pd.DataFrame(dict((spice, df_prep.keyFeatures.str.contains(spice, re.IGNORECASE))
                             for spice in keywords))
spice_df.head()

In [ ]:
df = df_prep[columns].merge(spice_df, how='outer', left_index=True, right_index=True)

df.info()

In [ ]:
df.head(30)

In [ ]:
print(len(df_orig))
old_length = len(df_orig)

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

In [ ]:
df['tenure.tenureType'] = df['tenure.tenureType'].astype('category')

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

In [ ]:
df.describe().T

In [ ]:
df.describe(include=['object', 'category'], exclude=[]).T

In [ ]:
%matplotlib inline
df.hist(bins=50, figsize=(20, 20))
plt.show()

<div id = 3 style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#b08888;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 2rem;
              color:white;
          text-align:center;
          margin:0 auto;
          ">
    IDENTIFY OUTLIERS


</h2>
</div>

In [ ]:
df.isna().sum()

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bedrooms'] > 5].sort_values('bedrooms', ascending=False)

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bedrooms'] > 10].sort_values('bedrooms', ascending=False)

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bathrooms'] > 5]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['longitude_deviation'] > 1]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['latitude_deviation'] > 1]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['nearestStation'] > 20]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['nearestStation'] > 4].sort_values('nearestStation', ascending=False)

In [ ]:
df = df[(df['bedrooms'] <= 10)]
df = df[df['bathrooms'] <= 5]
df = df[((df['longitude_deviation'] <= 1) & (df['latitude_deviation'] <= 1))]
df = df[(df['nearestStation'] <= 4)]

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

df.describe().T

In [ ]:
df.hist(bins=50, figsize=(20, 20))
plt.show()

In [ ]:
#sns.pairplot(df, diag_kind='kde')

In [ ]:
#df.isna().any()
df.isna().sum()

In [ ]:
df = df.dropna()
print(f"{old_length} ==> {len(df)}")
old_length = len(df)
df.describe()

In [ ]:
df_features = df.drop(LABEL, axis=1)
df_features

In [ ]:
use_one_hot_encoding_not_dummies = False

if use_one_hot_encoding_not_dummies:
    raise NotImplementedError('not implemented one hot encoding yet')

    #from sklearn.preprocessing import OneHotEncoder
    #tenure_encoder = OneHotEncoder()
    #df_tenure_type = tenure_encoder.fit_transform(df_features[['tenure.tenureType']])
else:
    for column in categories:
        df_features = pd.concat([df_features, pd.get_dummies(df_features[column], prefix=column, drop_first=True)],
                                axis=1)
        df_features.drop([column], axis=1, inplace=True)  # now drop the original column (you don't need it anymore),

features_corr = df_features.corr()
features_corr

In [ ]:
sns.set(font_scale=5)
#plt.figure(figsize=(50, 35))
plt.figure(figsize=(80, 80))
ax = sns.heatmap(features_corr, annot=True, fmt='.1f', cmap='coolwarm')

In [ ]:
data_corr = df.corr()
data_corr_dict = data_corr["Price"].sort_values(ascending=False).to_dict()
data_corr_dict

In [ ]:
#print the most positively correlated features
pos_corr = {k: v for k, v in data_corr_dict.items() if v > 0.1 and v != 1}
pos_corr

In [ ]:
#print the most negatively correlated features
neg_corr = {k: v for k, v in data_corr_dict.items() if v < -0.1}
neg_corr

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='bedrooms', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='latitude_deviation', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='longitude_deviation', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='latitude_deviation', y='longitude_deviation', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

<div id = 3 style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#b08888;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 2rem;
              color:white;
          text-align:center;
          margin:0 auto;
          ">
    Exploration for imputation


</h2>
</div>

In [ ]:
df['central_distance'] = (df['latitude_deviation'] ** 2 + df['longitude_deviation'] ** 2) ** 0.5
df

In [ ]:

plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='bedrooms', y='central_distance', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
#df.groupby(['tenure.tenureType','bedrooms']).agg({'bedrooms':['mean','min','max']})
df.groupby(['tenure.tenureType', 'bedrooms']).agg('count')